In [1]:
# !pip install -U torchdata
# !pip install -U spacy
# !python -m spacy download en_core_web_lg
# !python -m spacy download de_core_news_lg
# !pip install portalocker
! pip install accelerate -U
! pip install evaluate
! pip install sentence-transformers
! pip install SentencePiece
! pip install bert_score
! pip install --upgrade nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.8 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [2]:
#  import T5
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import numpy as np
import os

import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    DebertaTokenizer,
    DebertaForSequenceClassification,
    AutoConfig,
)

import logging
import evaluate
from evaluate import load
from datasets import load_dataset

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import T5Tokenizer,T5ForConditionalGeneration

import nltk
from nltk.translate.bleu_score import corpus_bleu

import matplotlib.pyplot as plt

from  transformers  import  AutoTokenizer, AutoModelWithLMHead

import csv
from torch.utils.data import DataLoader, Dataset
import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

device=""
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    print('Using CPU')

2024-04-25 05:15:34.557718: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 05:15:34.557816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 05:15:34.696972: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using GPU: Tesla P100-PCIE-16GB


In [3]:
PATH="/kaggle/input/webis-clickbait-22/"
OUTPATH="/kaggle/working/"
# PATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\"
# OUTPATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\output\\"
BATCH=8
T5_model = "MaRiOrOsSi/t5-base-finetuned-question-answering"
sep='[SEP]'

In [4]:
# data=pd.read_json(PATH+"train.jsonl",lines=True,encoding='utf-8')
# data.head()

In [5]:
def get_split_data(file_path,if_train=True):
    global label_encoder
    data=pd.read_json(file_path,lines=True,encoding='utf-8')


    y = data["spoiler"]
    Y=[]
    for i in y:
        Y.append(" ".join(i))
        
    new_data=data[['postText','targetParagraphs','targetTitle','targetDescription','targetKeywords']]
    dic=new_data.to_dict(orient='records')

    for i in dic:
        # print(i)
        i['postText']=i['postText'][0]
        targetparah=" ".join(i['targetParagraphs'])
        i['targetParagraphs']=targetparah
        # if(targetDescription is None or len(targetDescription)==0):
        #     targetDescription='none'
        #     desccount+=1
        # i['targetDescription']='targetDescription: '+targetDescription
        # targetkey=i['targetKeywords']
        # if(targetkey is None or len(targetkey)==0):
        #     targetkey='none'
        #     keycount+=1
        # i['targetKeywords']='targetKeywords: '+targetkey
        i['question']="question : "+i['postText']+"?"
        i['context']="context : "+i['targetParagraphs']
        del i['postText']
        del i['targetParagraphs']
        del i['targetTitle']
        del i['targetDescription']
        del i['targetKeywords']

    # print("desccount: ",desccount)
    # print("keycount: ",keycount)
    X_q = [i['question'] for i in dic]
    X_q=np.array(X_q)
    X_c = [i['context'] for i in dic]
    X_c=np.array(X_c)

#     return X_q[:100],X_c[:100],Y[:100]
    return X_q,X_c,Y

In [6]:
x_train_q,x_train_c,y_train = get_split_data(PATH+'train.jsonl')
x_test_q,x_test_c,y_test = get_split_data(PATH+'test.jsonl',False)
x_val_q,x_val_c,y_val = get_split_data(PATH+'validation.jsonl',False)

In [7]:
x_train_q.shape,x_train_c.shape,len(y_train),x_test_q.shape,x_test_c.shape,len(y_test),x_val_q.shape,x_val_c.shape,len(y_val)

((3200,), (3200,), 3200, (1000,), (1000,), 1000, (800,), (800,), 800)

In [8]:
# create pd dataframe
train_df = pd.DataFrame(list(zip(x_train_q, x_train_c, y_train)), columns =['question', 'context', 'spoiler'])
val_df = pd.DataFrame(list(zip(x_val_q, x_val_c, y_val)), columns =['question', 'context', 'spoiler'])
test_df = pd.DataFrame(list(zip(x_test_q, x_test_c, y_test)), columns =['question', 'context', 'spoiler'])


In [9]:
train_df["question"][0],train_df["context"][0],train_df["spoiler"][0]

('question : Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had Better Idea?',
 'context : It’ll be just like old times this weekend for Tom Brady and Wes Welker. Welker revealed Friday morning on a Miami radio station that he contacted Brady because he’ll be in town for Sunday’s game between the New England Patriots and Miami Dolphins at Gillette Stadium. It seemed like a perfect opportunity for the two to catch up. But Brady’s definition of "catching up" involves far more than just a meal. In fact, it involves some literal "catching" as the Patriots quarterback looks to stay sharp during his four-game Deflategate suspension. "I hit him up to do dinner Saturday night. He’s like, ‘I’m going to be flying in from Ann Arbor later (after the Michigan-Colorado football game), but how about that morning we go throw?’ " Welker said on WQAM, per The Boston Globe. "And I’m just sitting there, I’m like, ‘I was just thinking about dinner, but yeah, sure. I’ll get over there early and we

In [10]:
# t5
tokenizer = AutoTokenizer.from_pretrained(T5_model)
model = AutoModelWithLMHead.from_pretrained(T5_model)


tokenizer_config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1682: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [11]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

### convert to Huggingface dataset
train_data = Dataset(pa.Table.from_pandas(train_df))
val_data = Dataset(pa.Table.from_pandas(val_df))
test_data = Dataset(pa.Table.from_pandas(test_df))
train_data,val_data,test_data

(Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 3200
 }),
 Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 800
 }),
 Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 1000
 }))

In [12]:

train_data[0]

{'question': 'question : Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had Better Idea?',
 'context': 'context : It’ll be just like old times this weekend for Tom Brady and Wes Welker. Welker revealed Friday morning on a Miami radio station that he contacted Brady because he’ll be in town for Sunday’s game between the New England Patriots and Miami Dolphins at Gillette Stadium. It seemed like a perfect opportunity for the two to catch up. But Brady’s definition of "catching up" involves far more than just a meal. In fact, it involves some literal "catching" as the Patriots quarterback looks to stay sharp during his four-game Deflategate suspension. "I hit him up to do dinner Saturday night. He’s like, ‘I’m going to be flying in from Ann Arbor later (after the Michigan-Colorado football game), but how about that morning we go throw?’ " Welker said on WQAM, per The Boston Globe. "And I’m just sitting there, I’m like, ‘I was just thinking about dinner, but yeah, sure. I’ll get 

In [13]:
def preproc(data):
    q=[]
    c=[]
    a=[]
    # print(len(data))
    # print(len(data['question']))
    # print(len(data['context']))
    # print(len(data['spoiler']))
    for i in range(len(data['question'])):
        q.append(data['question'][i])
        c.append(data['context'][i])
        a.append(data['spoiler'][i])

    model_inputs=tokenizer(q,c,text_target=a,return_tensors='pt',padding=True,truncation=True,max_length=512)
    return model_inputs


In [14]:
# train_data=Dataset.from_pandas(train_df)
tokenized_train=train_data.map(preproc,batched=True,batch_size=BATCH)
tokenized_val=val_data.map(preproc,batched=True,batch_size=BATCH)
tokenized_test=test_data.map(preproc,batched=True,batch_size=BATCH)
tokenized_train,tokenized_val,tokenized_test

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

(Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 3200
 }),
 Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 800
 }),
 Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 1000
 }))

In [15]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=T5_model)

In [16]:
import wandb
api_key = "9963cf6219e451d47251ea34645181ada1b2526b"
wandb.login(key=api_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [17]:
model = AutoModelForSeq2SeqLM.from_pretrained(T5_model)
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [18]:
bertscore = load("bertscore")
meteor = evaluate.load("meteor")
bleu = evaluate.load("bleu")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [19]:

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    pred, ref = postprocess_text(decoded_preds, decoded_labels)

    # cal bleu, meteor and bert
    bleu_score = bleu.compute(predictions=pred, references=ref)
#     bleu1 = corpus_bleu(ref,pred,weights=(1,0,0,0))
#     bleu2 = corpus_bleu(ref,pred,weights=(0.5,0.5,0))
#     bleu3 = corpus_bleu(ref,pred,weights=(0.33,0.33,0.33,0)
#     bleu4 = corpus_bleu(ref,pred,weights=(0.25,0.25,0.25,0.25))

    print(bleu_score)
    meteor_score = meteor.compute(predictions=pred, references=ref)
    bertscore_score = bertscore.compute(predictions=pred, references=ref, lang="en")

    #  dict
    return {"blue":bleu_score["bleu"],
            "precisions_1":bleu_score["precisions"][0],
            "precisions_2":bleu_score["precisions"][1],
            "precisions_3":bleu_score["precisions"][2],
            "precisions_4":bleu_score["precisions"][3],
            "bp":bleu_score["brevity_penalty"],
            "meteor": meteor_score["meteor"], 
            "bertscore_f1": np.average(bertscore_score["f1"]), 
            "bertscore_p": np.average(bertscore_score["precision"]), 
            "bertscore_r": np.average(bertscore_score["recall"])}


In [20]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./T5",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH,
    per_device_eval_batch_size=BATCH,
    weight_decay=0.01,
    save_total_limit=4,
    num_train_epochs=12,
    predict_with_generate=True,
    save_strategy="epoch",
    fp16=True,
#     report_to="wandb",
#     logging_dir='./lolololol'

    # push_to_hub=True
    # load_best_model_at_end=True,
    # metric_for_best_model="bleu"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Currently logged in as: raghav21274 (ragha). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240425_051610-niguga11
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run radiant-surf-59
wandb: ⭐️ View project at https://wandb.ai/ragha/huggingface
wandb: 🚀 View run at https://wandb.ai/ragha/huggingface/runs/niguga11


Epoch,Training Loss,Validation Loss,Blue,Precisions 1,Precisions 2,Precisions 3,Precisions 4,Bp,Meteor,Bertscore F1,Bertscore P,Bertscore R
1,No log,0.766831,0.000000,0.152542,0.094340,0.062500,0.046512,0.000000,0.001874,0.006415,0.006407,0.006425
2,3.311000,0.599290,0.000000,0.314607,0.260274,0.213115,0.192308,0.000000,0.006742,0.017550,0.017408,0.017718
3,0.600000,0.503555,0.062156,0.296320,0.202128,0.175331,0.164808,0.304737,0.195580,0.786758,0.791670,0.783325
4,0.480500,0.481958,0.076517,0.313262,0.221067,0.194208,0.182464,0.343784,0.229059,0.840243,0.844867,0.837116
5,0.421100,0.469178,0.077281,0.338705,0.243658,0.214024,0.200046,0.316974,0.249580,0.862655,0.868187,0.858695
6,0.421100,0.462024,0.084304,0.342001,0.250908,0.221324,0.207148,0.336618,0.259458,0.867011,0.872061,0.863510
7,0.401100,0.455785,0.081909,0.352766,0.257857,0.227597,0.213648,0.317613,0.264978,0.870911,0.877081,0.866244
8,0.373400,0.452379,0.079587,0.359443,0.264463,0.233706,0.219639,0.301127,0.267540,0.871264,0.877501,0.866511
9,0.385500,0.449881,0.078480,0.360168,0.265776,0.235859,0.221609,0.295083,0.262636,0.871141,0.877734,0.866010
10,0.360800,0.449657,0.078871,0.360193,0.265680,0.235419,0.220465,0.297096,0.267128,0.873431,0.879877,0.868457


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 1.6983684112199046e-102, 'precisions': [0.15254237288135594, 0.09433962264150944, 0.0625, 0.046511627906976744], 'brevity_penalty': 2.111788186569837e-101, 'length_ratio': 0.0042952824694234126, 'translation_length': 59, 'reference_length': 13736}


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.07846506666552286, 'precisions': [0.35909383033419023, 0.2630899705014749, 0.23176420695504665, 0.2162807352590117], 'brevity_penalty': 0.29911089919627265, 'length_ratio': 0.45311589982527667, 'translation_length': 6224, 'reference_length': 13736}


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.07950560987402511, 'precisions': [0.359602755085696, 0.2643762630902076, 0.2336290663286861, 0.21878715814506539], 'brevity_penalty': 0.3011266763331528, 'length_ratio': 0.45449912638322654, 'translation_length': 6243, 'reference_length': 13736}


TrainOutput(global_step=4800, training_loss=0.7188734579086303, metrics={'train_runtime': 4860.9273, 'train_samples_per_second': 7.9, 'train_steps_per_second': 0.987, 'total_flos': 2.3383982997504e+16, 'train_loss': 0.7188734579086303, 'epoch': 12.0})

In [21]:
trainer.save_model("T5_overall")

In [22]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [23]:
results = trainer.evaluate(eval_dataset=tokenized_test)
print(results)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.0943040679199106, 'precisions': [0.36907988127500324, 0.26711111111111113, 0.23016683690840994, 0.21263926238955053], 'brevity_penalty': 0.3578127351720152, 'length_ratio': 0.4931585311525488, 'translation_length': 7749, 'reference_length': 15713}


{'eval_loss': 0.4351721405982971, 'eval_blue': 0.0943040679199106, 'eval_precisions_1': 0.36907988127500324, 'eval_precisions_2': 0.26711111111111113, 'eval_precisions_3': 0.23016683690840994, 'eval_precisions_4': 0.21263926238955053, 'eval_bp': 0.3578127351720152, 'eval_meteor': 0.3048531613928537, 'eval_bertscore_f1': 0.8788262942433357, 'eval_bertscore_p': 0.8851327450275421, 'eval_bertscore_r': 0.8738773688077927, 'eval_runtime': 108.3117, 'eval_samples_per_second': 9.233, 'eval_steps_per_second': 1.154, 'epoch': 12.0}


In [24]:
def calculate_bleu(bp,precisions):
    weights_1 = np.array([1,0,0,0])
    weights_2 = np.array([0.5,0.5,0,0])
    weights_3 = np.array([0.33,0.33,0.33,0])
    weights_4 = np.array([0.25,0.25,0.25,0.25])
    logp = np.log(precisions)
        
    res1 = bp*np.exp(np.dot(logp,weights_1))
    res2 = bp*np.exp(np.dot(logp,weights_2))
    res3 = bp*np.exp(np.dot(logp,weights_3))
    res4 = bp*np.exp(np.dot(logp,weights_4))
    
    return [res1,res2,res3,res4]

In [25]:
results = trainer.evaluate(eval_dataset=tokenized_train)
print(results)
calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

{'bleu': 0.12449386710806865, 'precisions': [0.4654080487413821, 0.3807181279021654, 0.35010049719665715, 0.33522117562894954], 'brevity_penalty': 0.3278366874946611, 'length_ratio': 0.47275965966155653, 'translation_length': 24948, 'reference_length': 52771}


{'eval_loss': 0.3083861172199249, 'eval_blue': 0.12449386710806865, 'eval_precisions_1': 0.4654080487413821, 'eval_precisions_2': 0.3807181279021654, 'eval_precisions_3': 0.35010049719665715, 'eval_precisions_4': 0.33522117562894954, 'eval_bp': 0.3278366874946611, 'eval_meteor': 0.3674890648321681, 'eval_bertscore_f1': 0.8915079071931541, 'eval_bertscore_p': 0.8985996587015689, 'eval_bertscore_r': 0.8858036621846259, 'eval_runtime': 347.1694, 'eval_samples_per_second': 9.217, 'eval_steps_per_second': 1.152, 'epoch': 12.0}


[0.15257783303272848,
 0.13799910693913825,
 0.13098619005052298,
 0.12449386710806866]

In [26]:
results = trainer.evaluate(eval_dataset=tokenized_val)
print(results)
calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

{'bleu': 0.07950560987402511, 'precisions': [0.359602755085696, 0.2643762630902076, 0.2336290663286861, 0.21878715814506539], 'brevity_penalty': 0.3011266763331528, 'length_ratio': 0.45449912638322654, 'translation_length': 6243, 'reference_length': 13736}
{'eval_loss': 0.44811272621154785, 'eval_blue': 0.07950560987402511, 'eval_precisions_1': 0.359602755085696, 'eval_precisions_2': 0.2643762630902076, 'eval_precisions_3': 0.2336290663286861, 'eval_precisions_4': 0.21878715814506539, 'eval_bp': 0.3011266763331528, 'eval_meteor': 0.26822740725318406, 'eval_bertscore_f1': 0.8738349495828152, 'eval_bertscore_p': 0.8802987527102232, 'eval_bertscore_r': 0.8688665181398392, 'eval_runtime': 87.4242, 'eval_samples_per_second': 9.151, 'eval_steps_per_second': 1.144, 'epoch': 12.0}


[0.1082859824392004,
 0.0928478743912692,
 0.0857270943655126,
 0.07950560987402511]

In [27]:
results = trainer.evaluate(eval_dataset=tokenized_test)
print(results)
calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

{'bleu': 0.0943040679199106, 'precisions': [0.36907988127500324, 0.26711111111111113, 0.23016683690840994, 0.21263926238955053], 'brevity_penalty': 0.3578127351720152, 'length_ratio': 0.4931585311525488, 'translation_length': 7749, 'reference_length': 15713}
{'eval_loss': 0.4351721405982971, 'eval_blue': 0.0943040679199106, 'eval_precisions_1': 0.36907988127500324, 'eval_precisions_2': 0.26711111111111113, 'eval_precisions_3': 0.23016683690840994, 'eval_precisions_4': 0.21263926238955053, 'eval_bp': 0.3578127351720152, 'eval_meteor': 0.3048531613928537, 'eval_bertscore_f1': 0.8788262942433357, 'eval_bertscore_p': 0.8851327450275421, 'eval_bertscore_r': 0.8738773688077927, 'eval_runtime': 107.6325, 'eval_samples_per_second': 9.291, 'eval_steps_per_second': 1.161, 'epoch': 12.0}


[0.13206148181597155,
 0.11234712337055032,
 0.10258596987085371,
 0.09430406791991058]

In [28]:
q='question : Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had Better Idea?'
c='context : It’ll be just like old times this weekend for Tom Brady and Wes Welker. Welker revealed Friday morning on a Miami radio station that he contacted Brady because he’ll be in town for Sunday’s game between the New England Patriots and Miami Dolphins at Gillette Stadium. It seemed like a perfect opportunity for the two to catch up. But Brady’s definition of "catching up" involves far more than just a meal. In fact, it involves some literal "catching" as the Patriots quarterback looks to stay sharp during his four-game Deflategate suspension. "I hit him up to do dinner Saturday night. He’s like, ‘I’m going to be flying in from Ann Arbor later (after the Michigan-Colorado football game), but how about that morning we go throw?’ " Welker said on WQAM, per The Boston Globe. "And I’m just sitting there, I’m like, ‘I was just thinking about dinner, but yeah, sure. I’ll get over there early and we can throw a little bit.’ " Welker was one of Brady’s favorite targets for six seasons from 2007 to 2012. It’s understandable him and Brady want to meet with both being in the same area. But Brady typically is all business during football season. Welker probably should have known what he was getting into when reaching out to his buddy. "That’s the only thing we really have planned," Welker said of his upcoming workout with Brady. "It’s just funny. I’m sitting there trying to have dinner. ‘Hey, get your ass up here and let’s go throw.’ I’m like, ‘Aw jeez, man.’ He’s going to have me running like 2-minute drills in his backyard or something." Maybe Brady will put a good word in for Welker down in Foxboro if the former Patriots wide receiver impresses him enough.'
enc=tokenizer(q+" "+c,return_tensors='pt',padding=True,truncation=True,max_length=512)
out=model.generate(input_ids=enc['input_ids'].to(device))

In [29]:
tokenizer.decode(out[0]),'how about that morning we go throw?'

('<pad> "That’s just funny. I’m sitting there trying to have dinner. ‘He',
 'how about that morning we go throw?')

In [30]:
import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/working/__notebook__.ipynb
/kaggle/working/T5_overall/spiece.model
/kaggle/working/T5_overall/training_args.bin
/kaggle/working/T5_overall/model.safetensors
/kaggle/working/T5_overall/generation_config.json
/kaggle/working/T5_overall/tokenizer.json
/kaggle/working/T5_overall/config.json
/kaggle/working/T5_overall/tokenizer_config.json
/kaggle/working/T5_overall/special_tokens_map.json
/kaggle/working/T5/runs/Apr25_05-16-09_64bf8f55a826/events.out.tfevents.1714027150.64bf8f55a826.24.1
/kaggle/working/T5/runs/Apr25_05-16-09_64bf8f55a826/events.out.tfevents.1714022170.64bf8f55a826.24.0
/kaggle/working/T5/checkpoint-4000/spiece.model
/kaggle/working/T5/checkpoint-4000/training_args.bin
/kaggle/working/T5/checkpoint-4000/model.safetensors
/kaggle/working/T5/checkpoint-4000/generation_config.json
/kaggle/working/T5/checkpoint-4000/trainer_state.json
/kaggle/working/T5/checkpoint-4000/optimizer.pt
/kaggle/working/T5/checkpoint-4000/tokenizer.json
/kaggle/working/T5/checkpoint-4000/con